교재의 코드를 가져와서 사용하고, 수정하면서 테스트 한 코드들입니다.
수정 과정에서 일부 동작하지 않는 코드들도 있습니다.

In [1]:
import cv2
from PIL import Image
import sys
import os
import numpy as np

current_dir = os.getcwd()
sys.path.append(current_dir)
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
image_folder_path = './G1020/Images/'  

In [4]:
import pandas as pd
df = pd.read_csv('G1020.csv')
image_files = df['imageID'].tolist()  # 이미지 파일 이름이 있는 열 이름
labels = df['binaryLabels'].values  # 레이블이 있는 열 이름

# 이미지 크기 설정
target_size = (28, 28)

# cv2로 이미지 읽고 전처리
def load_and_extract_hue(image_path, target_size):
    image = cv2.imread(image_path)  # 이미지 읽기 (기본 BGR 형식)
    if image is None:
        print(f"Image not found: {image_path}")
        return None
    image = cv2.resize(image, target_size)  # 이미지 크기 조정
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  # BGR을 HSV로 변환
    hue_channel = hsv_image[:, :, 0]  # Hue 채널만 추출
    hue_channel = hue_channel / 180.0  # Hue 값 정규화 (0~1 범위로 스케일링, Hue 범위는 0-179)
    return hue_channel

# 모든 이미지를 불러와서 리스트에 저장
images = [load_and_extract_hue(image_folder_path + img_path, target_size) for img_path in image_files]
images = np.array([img for img in images if img is not None])  # None 값 제거

# 차원 추가하여 (1020, 1, 128, 128) 형태로 변환
images = images[:, np.newaxis, :, :]

print(f"Processed Hue images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

Processed Hue images shape: (1020, 1, 28, 28)
Labels shape: (1020,)


In [6]:
from mymethod.neural_network import *
from common.trainer import Trainer
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.298728745272224
=== epoch:1, train acc:0.71, test acc:0.65 ===
train loss:2.2915672513341816
train loss:2.278192103247589
train loss:2.2611860287323093
train loss:2.2288854519747305
train loss:2.2021287573809127
train loss:2.1320700536686896
train loss:2.0693445921974463
train loss:1.9693877910185258
train loss:1.857416198242364
train loss:1.726076208850762
train loss:1.5490495904720016
train loss:1.421714440860163
train loss:1.1211308254016175
train loss:1.132373071517696
train loss:0.9153614355164637
train loss:0.6540820043891454
train loss:0.5305928676705182
train loss:0.6673070916095627
train loss:0.6826430451703822
train loss:0.2117629012697873
train loss:0.1147160907361311
train loss:1.1995831342767007
train loss:0.9977086903322581
train loss:0.8064557628032339
train loss:1.1979887524747976
train loss:0.32632353421200316
train loss:0.33527168362406956
train loss:0.37286250112090313
train loss:0.5474583312672959
train loss:0.49378366604410945
train loss:0.387927741128

In [12]:
class ConvNet7Layer:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = int((input_size - filter_size + 2 * filter_pad) / filter_stride + 1)
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))


        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b3'] = np.zeros(hidden_size)        
        self.params['W4'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b4'] = np.zeros(hidden_size)
        self.params['W5'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b5'] = np.zeros(hidden_size)
        self.params['W6'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b6'] = np.zeros(hidden_size)
        self.params['W7'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b7'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.layers['Relu3'] = Relu()  # 추가된 ReLU 활성화 함수
        self.layers['Affine3'] = Affine(self.params['W4'], self.params['b4'])
        self.layers['Relu4'] = Relu()  # 추가된 ReLU 활성화 함
        self.layers['Affine4'] = Affine(self.params['W5'], self.params['b5'])
        self.layers['Relu5'] = Relu()  # 추가된 ReLU 활성화 함
        self.layers['Affine5'] = Affine(self.params['W6'], self.params['b6'])
        self.layers['Relu6'] = Relu()  # 추가된 ReLU 활성화 함
        self.layers['Affine6'] = Affine(self.params['W7'], self.params['b7'])

        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3,4,5,6,7):
        # for idx in (1, 2, 3,4,5):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W4'], grads['b4'] = self.layers['Affine3'].dW, self.layers['Affine3'].db
        grads['W5'], grads['b5'] = self.layers['Affine4'].dW, self.layers['Affine4'].db
        grads['W6'], grads['b6'] = self.layers['Affine5'].dW, self.layers['Affine5'].db
        grads['W7'], grads['b7'] = self.layers['Affine6'].dW, self.layers['Affine6'].db


        return grads


In [13]:
max_epochs = 20

network = ConvNet7Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.3008114774041903
=== epoch:1, train acc:0.7, test acc:0.65 ===
train loss:2.2988904549713927
train loss:2.2969470171466773
train loss:2.2948219251030877
train loss:2.292427563758855
train loss:2.2904356052459898
train loss:2.2877042553616165
train loss:2.2843028112514205
train loss:2.2819834593741843
train loss:2.278728535281702
train loss:2.274476995767912
train loss:2.270079530444556
train loss:2.2656456872486475
train loss:2.26201989154609
train loss:2.25684617735912
train loss:2.2511047784432554
train loss:2.2447073577084025
train loss:2.236492987613217
train loss:2.2270199809842626
train loss:2.2186166993680247
train loss:2.208667449523956
train loss:2.194595446136657
train loss:2.1722494322061574
train loss:2.1495205757108704
train loss:2.121939319740249
train loss:2.060846425053188
train loss:2.018914764356996
train loss:1.8913538246663317
train loss:1.7272542997486073
train loss:1.5710305268491693
train loss:1.255144951366407
train loss:0.989104062369163
train loss

In [14]:
class ConvNet3Layer:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)


        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [15]:
max_epochs = 20

network = ConvNet3Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.2955387843687447
=== epoch:1, train acc:0.7, test acc:0.65 ===
train loss:2.2810683670432814
train loss:2.2593837199460607
train loss:2.2197764723097757
train loss:2.1478874392318335
train loss:2.0701371567095213
train loss:1.9480864399550586
train loss:1.787726669259166
train loss:1.5525014712680365
train loss:1.3375529063478897
train loss:1.1191912006098188
train loss:0.8837140680144815
train loss:0.8132085069479578
train loss:0.6379004091810669
train loss:0.7032713726106979
train loss:1.0079225347317413
train loss:0.6465900498169226
train loss:0.379905652326491
train loss:0.19155091173913907
train loss:0.8160813285587283
train loss:0.7996255301432403
train loss:0.8187666091826667
train loss:0.5537119595275705
train loss:0.5059697083668777
train loss:0.6918266952821859
train loss:0.6359812340155764
train loss:0.5690794769793784
train loss:0.5794678193117313
train loss:0.4380813741126507
train loss:0.7333851611363766
train loss:0.4930857716226247
train loss:0.548060587239

In [16]:
class ConvNet5Layer:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = int((input_size - filter_size + 2 * filter_pad) / filter_stride + 1)
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))


        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b3'] = np.zeros(hidden_size)        
        self.params['W4'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b4'] = np.zeros(hidden_size)
        self.params['W5'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b5'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.layers['Affine3'] = Affine(self.params['W4'], self.params['b4'])
        self.layers['Affine4'] = Affine(self.params['W5'], self.params['b5'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3,4,5):
        # for idx in (1, 2, 3,4,5):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W4'], grads['b4'] = self.layers['Affine3'].dW, self.layers['Affine3'].db
        grads['W5'], grads['b5'] = self.layers['Affine4'].dW, self.layers['Affine4'].db

        return grads

In [17]:
max_epochs = 20

network = ConvNet5Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.300298718590973
=== epoch:1, train acc:0.7, test acc:0.65 ===
train loss:2.297649240616901
train loss:2.2954909359783167
train loss:2.29121456549611
train loss:2.285370069035045
train loss:2.275604011666046
train loss:2.252248165203401
train loss:2.2202715478866635
train loss:2.175352080370364
train loss:2.044328361775077
train loss:1.8948495206536413
train loss:1.7007444769644693
train loss:1.374749340073309
train loss:0.989747188805946
train loss:0.7353986025497822
train loss:0.6354294920267323
train loss:0.4999146224374235
train loss:0.5780262115786929
train loss:1.2471999051716132
train loss:0.29939264819476896
train loss:0.7836296373345997
train loss:0.6168812337884446
train loss:0.7120390510488065
train loss:0.7348575934042839
train loss:0.768463046653386
train loss:0.9126951725947405
train loss:0.6977971622430833
train loss:0.7017946147292363
train loss:0.685082671986686
train loss:0.3527086817808157
train loss:0.858737118513262
train loss:0.5569937020553581
train l

In [18]:
class ConvNet7Layer:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = int((input_size - filter_size + 2 * filter_pad) / filter_stride + 1)
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))


        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b3'] = np.zeros(hidden_size)        
        self.params['W4'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b4'] = np.zeros(hidden_size)
        self.params['W5'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b5'] = np.zeros(hidden_size)
        self.params['W6'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b6'] = np.zeros(hidden_size)
        self.params['W7'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b7'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.layers['Affine3'] = Affine(self.params['W4'], self.params['b4'])
        self.layers['Affine4'] = Affine(self.params['W5'], self.params['b5'])
        self.layers['Affine5'] = Affine(self.params['W6'], self.params['b6'])
        self.layers['Affine6'] = Affine(self.params['W7'], self.params['b7'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3,4,5,6,7):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W4'], grads['b4'] = self.layers['Affine3'].dW, self.layers['Affine3'].db
        grads['W5'], grads['b5'] = self.layers['Affine4'].dW, self.layers['Affine4'].db
        grads['W6'], grads['b6'] = self.layers['Affine5'].dW, self.layers['Affine5'].db
        grads['W7'], grads['b7'] = self.layers['Affine6'].dW, self.layers['Affine6'].db


        return grads

In [19]:
max_epochs = 20

network = ConvNet7Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.3002388202767476
=== epoch:1, train acc:0.7, test acc:0.65 ===
train loss:2.2975553834244855
train loss:2.2943938485468225
train loss:2.290904239149401
train loss:2.287286064204815
train loss:2.280722736408403
train loss:2.2747281377785966
train loss:2.267430130812123
train loss:2.258318875960108
train loss:2.241985632134329
train loss:2.2193028800375183
train loss:2.1863480716891512
train loss:2.127819405983845
train loss:2.0230786165282906
train loss:1.8323999510006694
train loss:1.5236624979588753
train loss:1.0967064100013422
train loss:0.7665785248121592
train loss:0.6154160675781347
train loss:0.3394318938919303
train loss:0.9254815148079876
train loss:0.8476672688021075
train loss:0.7997057059280236
train loss:0.5263935763660894
train loss:0.6214579338024937
train loss:0.5853496695225062
train loss:0.6634001066307305
train loss:0.681208794597784
train loss:0.6690378741335212
train loss:0.6662754616141012
train loss:0.7275082809109195
train loss:0.4874266004481037
tr

In [20]:
class ConvNet7Layer:
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = int((input_size - filter_size + 2 * filter_pad) / filter_stride + 1)
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))


        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b3'] = np.zeros(hidden_size)        
        self.params['W4'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b4'] = np.zeros(hidden_size)
        self.params['W5'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b5'] = np.zeros(hidden_size)
        self.params['W6'] = weight_init_std * np.random.randn(hidden_size, hidden_size)
        self.params['b6'] = np.zeros(hidden_size)
        self.params['W7'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b7'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Sigmoid()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Sigmoid()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.layers['Relu3'] = Sigmoid()  # 추가된 ReLU 활성화 함수
        self.layers['Affine3'] = Affine(self.params['W4'], self.params['b4'])
        self.layers['Relu4'] = Sigmoid()  # 추가된 ReLU 활성화 함
        self.layers['Affine4'] = Affine(self.params['W5'], self.params['b5'])
        self.layers['Relu5'] = Sigmoid()  # 추가된 ReLU 활성화 함
        self.layers['Affine5'] = Affine(self.params['W6'], self.params['b6'])
        self.layers['Relu6'] = Sigmoid()  # 추가된 ReLU 활성화 함
        self.layers['Affine6'] = Affine(self.params['W7'], self.params['b7'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3,4,5,6,7):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
        grads['W4'], grads['b4'] = self.layers['Affine3'].dW, self.layers['Affine3'].db
        grads['W5'], grads['b5'] = self.layers['Affine4'].dW, self.layers['Affine4'].db
        grads['W6'], grads['b6'] = self.layers['Affine5'].dW, self.layers['Affine5'].db
        grads['W7'], grads['b7'] = self.layers['Affine6'].dW, self.layers['Affine6'].db


        return grads

In [21]:
max_epochs = 20

network = ConvNet7Layer(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.2622155579089585
=== epoch:1, train acc:0.7, test acc:0.65 ===
train loss:2.1736192992844168
train loss:2.1029727597880337
train loss:2.0057717025064528
train loss:1.9480715768775838
train loss:1.8508828244554514
train loss:1.7817081390428862
train loss:1.6753632473199478
train loss:1.6411566986219852
train loss:1.52889883815578
train loss:1.4488426995780692
train loss:1.417025681920395
train loss:1.3188363723436123
train loss:1.2461335629965866
train loss:1.1451749963183817
train loss:1.074657208422506
train loss:0.9656456531590282
train loss:1.0350991660194684
train loss:0.9871741827847605
train loss:0.7193608671442777
train loss:0.9123668591209491
train loss:0.5988821258064481
train loss:0.7841482775026474
train loss:0.9312704746849763
train loss:0.6479351114536934
train loss:0.9156563853968163
train loss:0.6101052479899307
train loss:0.6997147598998769
train loss:0.9008610578084179
train loss:0.7869142540576888
train loss:1.0796620182669405
train loss:0.667028197001256

In [22]:
class DeepConvNet:
    def __init__(self, input_dim=(1, 28, 28), #필터 수 조절해서 성능 조절
                 conv_param_1 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_2 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_3 = {'filter_num':32, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_4 = {'filter_num':32, 'filter_size':3, 'pad':2, 'stride':1},
                 conv_param_5 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 conv_param_6 = {'filter_num':64, 'filter_size':3, 'pad':1, 'stride':1},
                 hidden_size=50, output_size=10):
        # 가중치 초기화===========
        # 각 층의 뉴런 하나당 앞 층의 몇 개 뉴런과 연결되는가（TODO: 자동 계산되게 바꿀 것）
        pre_node_nums = np.array([1*3*3, 16*3*3, 16*3*3, 32*3*3, 32*3*3, 64*3*3, 64*4*4, hidden_size])
        wight_init_scales = np.sqrt(2.0 / pre_node_nums)  # ReLU를 사용할 때의 권장 초깃값
        
        self.params = {}
        pre_channel_num = input_dim[0]
        for idx, conv_param in enumerate([conv_param_1, conv_param_2, conv_param_3, conv_param_4, conv_param_5, conv_param_6]):
            self.params['W' + str(idx+1)] = wight_init_scales[idx] * np.random.randn(conv_param['filter_num'], pre_channel_num, conv_param['filter_size'], conv_param['filter_size'])
            self.params['b' + str(idx+1)] = np.zeros(conv_param['filter_num'])
            pre_channel_num = conv_param['filter_num']
        self.params['W7'] = wight_init_scales[6] * np.random.randn(64*4*4, hidden_size)
        self.params['b7'] = np.zeros(hidden_size)
        self.params['W8'] = wight_init_scales[7] * np.random.randn(hidden_size, output_size)
        self.params['b8'] = np.zeros(output_size)

        # 계층 생성===========
        self.layers = []
        self.layers.append(Convolution(self.params['W1'], self.params['b1'], 
                           conv_param_1['stride'], conv_param_1['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W2'], self.params['b2'], 
                           conv_param_2['stride'], conv_param_2['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W3'], self.params['b3'], 
                           conv_param_3['stride'], conv_param_3['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W4'], self.params['b4'],
                           conv_param_4['stride'], conv_param_4['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Convolution(self.params['W5'], self.params['b5'],
                           conv_param_5['stride'], conv_param_5['pad']))
        self.layers.append(Relu())
        self.layers.append(Convolution(self.params['W6'], self.params['b6'],
                           conv_param_6['stride'], conv_param_6['pad']))
        self.layers.append(Relu())
        self.layers.append(Pooling(pool_h=2, pool_w=2, stride=2))
        self.layers.append(Affine(self.params['W7'], self.params['b7']))
        self.layers.append(Relu())
        self.layers.append(Dropout(0.5))
        self.layers.append(Affine(self.params['W8'], self.params['b8']))
        self.layers.append(Dropout(0.5))
        
        self.last_layer = SoftmaxWithLoss()

    def predict(self, x, train_flg=False):
        for layer in self.layers:
            if isinstance(layer, Dropout):
                x = layer.forward(x, train_flg)
            else:
                x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x, train_flg=True)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)

        acc = 0.0

        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx, train_flg=False)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt)

        return acc / x.shape[0]

    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        tmp_layers = self.layers.copy()
        tmp_layers.reverse()
        for layer in tmp_layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            grads['W' + str(i+1)] = self.layers[layer_idx].dW
            grads['b' + str(i+1)] = self.layers[layer_idx].db

        return grads

    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, layer_idx in enumerate((0, 2, 5, 7, 10, 12, 15, 18)):
            self.layers[layer_idx].W = self.params['W' + str(i+1)]
            self.layers[layer_idx].b = self.params['b' + str(i+1)]            

In [24]:
max_epochs = 20

network = DeepConvNet()
trainer = Trainer(network, x_train, y_train, x_test, y_test,
                  epochs=max_epochs, mini_batch_size=10,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=100)
trainer.train()

train loss:2.0719775451053626
=== epoch:1, train acc:0.7, test acc:0.61 ===
train loss:2.1800809121178353
train loss:1.9636100748377714
train loss:1.6551257925811629
train loss:1.574160530305424
train loss:1.7821058609378855
train loss:0.8553402525695274
train loss:2.1451963192932526
train loss:2.162223591559808
train loss:0.719055386003319
train loss:1.6434724438415773
train loss:1.8630841875674435
train loss:1.3837112785090953
train loss:2.216024642472302
train loss:2.0179047992615704
train loss:1.557615721347896
train loss:1.2609492868045327
train loss:1.5950664293010963
train loss:1.401170570339697
train loss:2.1427611169659153
train loss:1.6595075377099076
train loss:1.7673580901603443
train loss:1.7365094842220636
train loss:1.7322291971250217
train loss:1.7269642358945316
train loss:1.6702431889401486
train loss:2.4638170623602433
train loss:1.4896966018586368
train loss:1.523643921940315
train loss:1.5996065869705616
train loss:1.317800304035552
train loss:1.6333577462142195
tr